In [5]:
import os
import pandas as pd
from Analysis import Analysis 
from getSignals import getSignals
from Plot import Plot

In [4]:
data_file = 'BTC_15'

In [2]:
def recallNum(data_file,number):
    data_file_ent = []
    for file in os.listdir(data_file):
        if file.endswith('.csv'):
            data_file_ent.append(file)
    for file in data_file_ent:
        if str(number) in file:
            df = pd.read_csv(data_file+'/'+'BTC'+str(number)+'.csv')
    return df